In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")

In [83]:
import numpy as np
import scipy
from misc import *
import matplotlib.pyplot as plt
from moses_simple import moses_move
from moses_variational import moses_move as mm_var, var_A as mm_var_A
from random_mps import random_mps_uniform
from state_approximation import mps2mpo

In [87]:

def var_Lambda_1site(Psi, A, Lambda, Lp_list=None):
    L = len(Psi)

    if Lp_list is None:
        Lp = np.zeros([1, 1, 1])
        Lp[0, 0, 0] = 1.
        Lp_list = [Lp]

        for i in range(L):
            Lp = np.tensordot(Lp, Psi[i], axes=(0, 2))
            Lp = np.tensordot(Lp, A[i], axes=([1, 2], [2, 0]))
            Lp = np.tensordot(Lp, Lambda[i], axes=([0, 3, 1], [2, 0, 1]))
            Lp = Lp.transpose([0, 2, 1])
            Lp_list.append(Lp)

    for i in range(2):
        Rp = np.zeros([1, 1, 1])
        Rp[0, 0, 0] = 1.
        Rp_list = [Rp]
        Lambdap = [[] for i in range(L)]

        for i in range(L - 1, -1, -1):
            Rp = np.tensordot(Rp, Psi[i], axes=(0, 3))
            Rp = np.tensordot(Rp, A[i], axes=([1, 2], [3, 0]))

            theta = np.tensordot(Rp, Lp_list[i], axes=([2, 4], [0, 2]))
            theta = theta.transpose(2, 1, 3, 0)

            d1, d2, chi1, chi2 = theta.shape
            #Q,R = np.linalg.qr(theta.transpose(0,1,3,2).reshape(d1*d2*chi2,chi1))
            Q, Y, Z = np.linalg.svd(theta.transpose(0, 1, 3, 2).reshape(d1 * d2 * chi2, chi1),full_matrices=False)
            Lambdap[i] = Q.reshape(d1, d2, chi2, -1).transpose(0, 1, 3, 2)
            Rp = np.tensordot(Rp, Lambdap[i], axes=([0, 1, 3], [3, 1, 0]))
            Rp = Rp.transpose(0, 2, 1)
            Rp_list.append(Rp)

        A = mps_invert(A)
        Psi = mps_invert(Psi)
        Lambda = mps_invert(Lambdap)
        Lp_list = Rp_list
    return Lambda, Lp_list

def var_A(Psi, A, Lambda, Lp_list=None):
    L = len(Psi)
    if Lp_list is None:
        Lp = np.zeros([1, 1, 1])
        Lp[0, 0, 0] = 1.
        Lp_list = [Lp]

        for i in range(L):
            Lp = np.tensordot(Lp, Psi[i], axes=(0, 2))
            Lp = np.tensordot(Lp, A[i].conj(), axes=([1, 2], [2, 0]))
            Lp = np.tensordot(Lp, Lambda[i].conj(), axes=([0, 3, 1], [2, 0, 1]))
            Lp = Lp.transpose([0, 2, 1])
            Lp_list.append(Lp)

    Rp = np.zeros([1, 1, 1])
    Rp[0, 0, 0] = 1.
    Rp_list = [Rp]
    Ap = [[] for i in range(L)]

    for i in range(L - 1, -1, -1):
        Rp = np.tensordot(Rp, Psi[i], axes=(0, 3))
        Rp = np.tensordot(Rp, Lambda[i].conj(), axes=([0, 3], [3, 1]))
        theta = np.tensordot(Rp, Lp_list[i], axes=([2, 4], [0, 1]))
        theta = theta.transpose(1, 3, 2, 0)

        chiL, chiD, chiR, chiU = theta.shape
        #X, s, Y = np.linalg.svd(theta.reshape(chiL * chiD, chiR * chiU),full_matrices=False)
        #Ap[i] = np.dot(X, Y).reshape(chiL, chiD, chiR, chiU).transpose(0, 2, 1, 3)
        Ap[i] = scipy.linalg.polar(theta.reshape(chiL*chiD, chiR*chiU))[0].reshape(chiL, chiD, chiR, chiU).transpose(0,2,1,3)
        Rp = np.tensordot(Rp, Ap[i].conj(), axes=([0, 1, 3], [3, 0, 1]))
        Rp_list.append(Rp)

    return Ap, Lp_list

def split(Psi,A,Lambda, truncation_par={'bond_dimensions': {'eta_max': 4,'chi_max': 4}}, N=10):

    if A == None or Lambda == None:
        A,Lambda = init_A_Lambda(Psi, truncation_par)

    Lp_list = None
    
    for i in range(N):
        A, Lp_list = var_A(Psi, A, Lambda, Lp_list=Lp_list)
        #Lambda, Lp_list = var_Lambda_1site(Psi, A, Lambda)
        
    return A, Lambda

In [88]:
L = 10
Psi_real = mps2mpo(random_mps_uniform(L, chi=32))
Psi_imag = mps2mpo(random_mps_uniform(L, chi=32))
Psi = [Psi_real[i] + 1.j * Psi_imag[i] for i in range(L)]
norm = mps_overlap(Psi, Psi)
Psi[0] /= np.sqrt(norm)
print(mps_overlap(Psi, Psi))

(1+0j)


In [89]:
truncation_par = dict(bond_dimensions=dict(eta_max=16, chi_max=100), p_trunc=0.0)
A, Lambda = moses_move(Psi, truncation_par)
A, Lambda = split(Psi, A, Lambda, truncation_par)
#A, Lambda, _ = mm_var(Psi, A, Lambda, truncation_par)
np.linalg.norm(mps_overlap(Psi, mpo_on_mpo(A, Lambda)))

0.8929909744790413